In [1]:
import sys

sys.path.append( '../slevelsutil/' )

import df_util, file_util
from Instance import Instance

### 2) Loading the instance data

In [2]:
filter_instance = {}#  {"instance_name":"SCENARIO"} #, "customer_segmentation":"X", "service_rate": "S1"}
test_case = "standard_vs_enforce_BB"
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/day/enforce_sl_all_hierachical_scenarios_150_plus.json"
data_dict_path = "../data/dictionary/request_track_data_dictionary_plus.json"

instance_case = Instance(test_case, instance_settings_path, data_dict_path)

import pandas as pd
from pprint import pprint

instance_case.category_sq_class = pd.api.types.CategoricalDtype(categories=instance_case.dict_sl_class.values(), ordered=True)
instance_case.category_fleet = pd.api.types.CategoricalDtype(categories=instance_case.dict_fleet.values(), ordered=True)

# Pickup deadline status (Was user chosen to have its pk deadline extended?)
dict_sl_status = instance_case.dict_sl_tier


### 2) Loading the instance data

In [3]:
instances_dic = file_util.load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]

print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

########### INSTANCE SETTINGS ##################################################
{'adjacency_matrix_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/network_info/adjacency_matrix.csv',
 'distances_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_m.csv',
 'durations_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_duration_s.csv',
 'hiring_config': [{'name': 'method_hire_from_regional_center'}],
 'instance_description': 'Tests cases currently used in paper (with more '
                         'flexible time cuts and larger graphs), 1h from 18h, '
                         'region centers from max waiting (150)',
 'instance_name': 'PLUS',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/hour_enforce_sl_plus/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_durat

### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output =
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [4]:
print(instance_case.dict_sl_class)

{'A': 'Business', 'B': 'Standard', 'C': 'Low-cost'}


### Aggregate results (folder request_track)

* `earliest`
* `id` = 1, 2, 3, ..., #USERS
* `class` = A, B, C
* `delay_pk`
* `ride_delay`
* `pk_time`
* `dp_time`
* `id_from` = Network id
* `id_to` = Network id
* `dist` = trip(id_from, id_to) in seconds
* `service` = {FLEET, FREELANCE}
* `service_level` = {FIRST, SECOND}

In [5]:
import pandas as pd

def get_request_track_dic(path_experiment, name_experiment):

    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)

    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)

    aggfunc = {"delay_pk" : ['mean', 'count', 'max']}

    dfp = df.pivot_table(index="class", columns="service_level", aggfunc=aggfunc, values=["delay_pk"])

    return dfp

### Processing all instances in folder (mean time)

In [6]:
from collections import defaultdict
import math


request_log_folder = result_folder + "request_track/"

# Get all instances in folder
instance_file_names = file_util.read_files_from_folder(request_log_folder)

def process_instance(file_name, instance_attribute, instance_attribute_count, instance_attribute_count_percentage, filter_instance = None, exclude_instance = None):

    instance, extension = file_name.split(".")

    label_setting_dic = instances_dic["labels"]

    # Instance settings
    instance_settings = Instance.get_instance_settings(label_setting_dic, instance)

    # Filter fixed fleet instances
    #if instance_settings.get('service_rate', '-') == '-':
    #    continue

    # Filter instance label
    if filter_instance:
        for i_attribute, i_filter in filter_instance.items():
            if instance_settings[i_attribute] != i_filter:
                print(f"Filtering {instance_settings[i_attribute]} {i_filter}")
                return

    # Filter instance label
    if exclude_instance:
        for i_attribute, i_filter in exclude_instance.items():
            if instance_settings[i_attribute] == i_filter:
                return

    print("  - Processing", instance)

    # Load results
    experiment_file = "{}{}.csv".format(request_log_folder, instance)
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)

    for a in ["service_rate", "contract_duration", "customer_segmentation", "matching", "maximal_hiring_delay"]:
        column = ('', '', a)
        # When no service rate is available, instance refers to baseline
        instance_attribute[column].append(instance_settings.get(a, "-"))
        instance_attribute_count_percentage[column].append(instance_settings.get(a, "-"))
        instance_attribute_count[column].append(instance_settings.get(a, "-"))


    # Convert denied to freelance to show denied under same header of the initial fleet
    df.loc[(df['service']=='DENIED'), 'service'] = 'FLEET'

    # Get total number of requests
    total = len(df)

    for sq_class in ['A', 'B', 'C']:

        subtotal_class = len(df[df['class'] == sq_class])
        for fleet in ['FLEET', 'FREELANCE']:
            for sl_status in ["FIRST", "SECOND"]:
                # Categories leading to max, mean and count
                super_indexes = (instance_case.dict_sl_class[sq_class], instance_case.dict_fleet[fleet], dict_sl_status[sl_status])

                # Filter dataframe
                filter_sq_sl = (df['service'] == fleet) & (df['class'] == sq_class) & (df['service_level'] == sl_status)
                df_filtered = df.loc[filter_sq_sl]["delay_pk"]
                mean_pk = df_filtered.mean()
                #max_pk = df_filtered.max()
                #count = df_filtered.count()

                # Format data
                # mean_pk_min = ("{:.1f}".format(mean_pk/60) if not math.isnan(mean_pk) else "-")
                mean_pk_min = (mean_pk if not math.isnan(mean_pk) else 0)
                #max_pk_min = ("{:.1f}".format(max_pk/60) if not math.isnan(max_pk) else "-")

                #count = ("{:.2%}".format(count/total) if not (math.isnan(count) or count==0) else "-")

                #instance_attribute[(*super_indexes, 'Max.')].append(max_pk_min)
                instance_attribute[super_indexes].append(mean_pk_min)
                #instance_attribute[(*super_indexes, 'Count')].append(count)



                count = df_filtered.count()
                instance_attribute_count[super_indexes].append(count)

                # Format data
                # mean_pk_min = ("{:.1f}".format(mean_pk/60) if not math.isnan(mean_pk) else "-")
                # mean_pk_min = ("{:.1f}".format(int(mean_pk)) if not math.isnan(mean_pk) else "-")
                #max_pk_min = ("{:.1f}".format(max_pk/60) if not math.isnan(max_pk) else "-")
                # subtotal_class+=count
                # count = (count/total if not (math.isnan(count) or count==0) else 0)
                count = (count/subtotal_class if not (math.isnan(count) or count==0) else 0)
                instance_attribute_count_percentage[super_indexes].append(count)

                #instance_attribute[(*super_indexes, 'Max.')].append(max_pk_min)
                # instance_attribute[(*super_indexes, 'Mean.')].append(mean_pk_min)

            # Adding total
            subtotal_index = (instance_case.dict_sl_class[sq_class], instance_case.dict_fleet[fleet], 'Total')

            count_fleet = len(df.loc[(df['service'] == fleet) & (df['class'] == sq_class)])
            instance_attribute_count[subtotal_index].append(count_fleet)

            count_fleet = (count_fleet/subtotal_class if not (math.isnan(count_fleet) or count_fleet==0) else 0)
            instance_attribute_count_percentage[subtotal_index].append(count_fleet)

instance_attribute = defaultdict(list)
instance_attribute_count = defaultdict(list)
instance_attribute_count_percentage = defaultdict(list)

for file_name in instance_file_names:
   process_instance(file_name, instance_attribute, instance_attribute_count, instance_attribute_count_percentage, filter_instance=filter_instance)

Reading 27 files from 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/hour_enforce_sl_plus/request_track/'.
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-JAVIER
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S0_VH_SD_RE-OP_OPT-JAVIERSL
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-JAVIERSL
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S8_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S8_VH_SD_RE-OP_OPT-JAVIERSL
  - Processing IN-PLUS_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S9_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-PLUS_HC-150_BA-30_ST-360

### Raw table (no labels and cell formatting)

In [7]:
raw_table = pd.DataFrame.from_dict(instance_attribute)
raw_table

\
                                                                    
   service_rate contract_duration customer_segmentation  matching   
0            S0                 0                    AA      ERTV   
1            S0                 0                    AA    JAVIER   
2            S0                 0                    AA  JAVIERSL   
3           S10                 0                    AA      ERTV   
4           S10                 0                    AA  JAVIERSL   
5            S8                 0                    AA      ERTV   
6            S8                 0                    AA  JAVIERSL   
7            S9                 0                    AA      ERTV   
8            S9                 0                    AA  JAVIERSL   
9            S0                 0                    BB      ERTV   
10           S0                 0                    BB    JAVIER   
11           S0                 0                    BB  JAVIERSL   
12          S10                 0                    BB      ERTV   
13          S10                 0                    BB  JAVIERSL   
14           S8                 0                    BB      ERTV   
15           S8                 0                    BB  JAVIERSL   
16           S9                 0                    BB      ERTV   
17           S9                 0                    BB  JAVIERSL   
18           S0                 0                    CC      ERTV   
19           S0                 0                    CC    JAVIER   
20           S0                 0                    CC  JAVIERSL   
21          S10                 0                    CC      ERTV   
22          S10                 0                    CC  JAVIERSL   
23           S8                 0                    CC      ERTV   
24           S8                 0                    CC  JAVIERSL   
25           S9                 0                    CC      ERTV   
26           S9                 0                    CC  JAVIERSL   

                           Business                                   \
                            Company               Freelance            
   maximal_hiring_delay    Achieved    Violated    Achieved Violated   
0                   150  121.734213    0.000000  104.987271        0   
1                   150   84.172585  274.004359    0.000000        0   
2                   150  124.367444  246.997964    0.000000        0   
3                   150  122.561251    0.000000  105.441571        0   
4                   150  124.055526  246.038527    0.000000        0   
5                   150  121.277026  361.639121  103.846850        0   
6                   150  125.179432  253.230950    0.000000        0   
7                   150  121.862102  354.041126  104.790435        0   
8                   150  123.119914  250.510624    0.000000        0   
9                   150   97.779806    0.000000  105.421756        0   
10                  150   81.832203  363.679312    0.000000        0   
11                  150  129.020588  312.898734    0.000000        0   
12                  150   97.234518    0.000000  105.594747        0   
13                  150  128.734922  328.393665    0.000000        0   
14                  150  120.609785  369.802260  112.177546        0   
15                  150  128.290092  335.385580    0.000000        0   
16                  150  116.422235  355.894118  109.267640        0   
17                  150  128.417615  326.558594    0.000000        0   
18                  150   96.752247    0.000000  106.945674        0   
19                  150   82.688581  361.529135    0.000000        0   
20                  150  124.828133  278.732919    0.000000        0   
21                  150   95.157655    0.000000  106.898077        0   
22                  150  125.357143  293.018127    0.000000        0   
23                  150  123.092504  372.223958  112.828431        0   
24                  150  124.855346  359.775068    0

### Table: What is the service level (pickup delay) of the users lying outside SQ-class service rate?

In [8]:
def create_table(instance_attribute):


    # Build data frame from dictionary
    a = pd.DataFrame.from_dict(instance_attribute)

    key_sr = ('','',  'service_rate') # (S1, S2, S3)
    key_cs = ('','',  'customer_segmentation') # (A, AA, BB, etc.)
    key_cd = ('','',  'contract_duration') #(0, 3600, 18000)
    key_me = ('','',  'matching') #(JAVIER, JAVIERSL, ERTV)
    key_hc = ('','',  'maximal_hiring_delay') #(0, 3600, 18000)

    indexes = [
        key_cs,
        key_cd,
        key_sr,
        key_me,
        key_hc
    ]
    # Filtering data (only mixed segmentations scenarios will be used)
    # a = a[a[key_cs].isin(["AA", "BB", "CC"])]

    # Renaming data and applying aliases
    a[key_cs] = a[key_cs].map(lambda e:instance_case.dict_segmentation[e])
    a[key_cs] = a[key_cs].astype(instance_case.category_segmentation)

    a[key_cd] = a[key_cd].map(lambda e:instance_case.dict_contract_duration[e])
    a[key_cd] = a[key_cd].astype(instance_case.category_contract_duration)

    a[key_sr] = a[key_sr].map(lambda e:instance_case.dict_service_rate[e])
    a[key_sr] = a[key_sr].astype(instance_case.category_service_rate)

    a[key_me] = a[key_me].map(lambda e:instance_case.dict_method[e])
    a[key_me] = a[key_me].astype(instance_case.category_method)

    a[key_hc] = a[key_hc].map(lambda e:instance_case.dict_maximal_hiring_delay[e])
    a[key_hc] = a[key_hc].astype(instance_case.category_maximal_hiring_delay)

    a = a.sort_values(by=indexes)
    a = a.set_index(indexes)

    # Changing table column index names
    a.index.names = [
        instance_case.headers["user_base"],
        instance_case.headers["contract_duration"],
        instance_case.headers["service_rate"],
        instance_case.headers["method"],
        instance_case.headers["maximal_hiring_delay"]
    ]

    # Transforming to minutes
    #a = a.applymap(lambda e: ('{:.2f}'.format(float(e)/60) if e != '-' else e))
    return a


### Table delay

In [9]:
instance_attribute

defaultdict(list,
            {('', '', 'service_rate'): ['S0',
              'S0',
              'S0',
              'S10',
              'S10',
              'S8',
              'S8',
              'S9',
              'S9',
              'S0',
              'S0',
              'S0',
              'S10',
              'S10',
              'S8',
              'S8',
              'S9',
              'S9',
              'S0',
              'S0',
              'S0',
              'S10',
              'S10',
              'S8',
              'S8',
              'S9',
              'S9'],
             ('', '', 'contract_duration'): ['0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0',
              '0'

In [10]:
df_delay = create_table(instance_attribute)
df_delay = df_util.format_as_time(df_delay)
df_delay

Business  \
                                                                                  Company   
                                                                                 Achieved   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                        01:24   
                                         Enforce SL        3                        02:04   
                                         Enforce SL + Hire 3                        02:01   
                            80%          Enforce SL        3                        02:05   
                                         Enforce SL + Hire 3                        02:01   
                            90%          Enforce SL        3                        02:03   
                                         Enforce SL + Hire 3                        02:01   
                            100%         Enforce SL        3                        02:04   
                                         Enforce SL + Hire 3                        02:02   
S+        Single-ride       0%           Min. waiting      3                        01:21   
                                         Enforce SL        3                        02:09   
                                         Enforce SL + Hire 3                        01:37   
                            80%          Enforce SL        3                        02:08   
                                         Enforce SL + Hire 3                        02:00   
                            90%          Enforce SL        3                        02:08   
                                         Enforce SL + Hire 3                        01:56   
                            100%         Enforce SL        3                        02:08   
                                         Enforce SL + Hire 3                        01:37   
L+        Single-ride       0%           Min. waiting      3                        01:22   
                                         Enforce SL        3                        02:04   
                                         Enforce SL + Hire 3                        01:36   
                            80%          Enforce SL        3                        02:04   
                                         Enforce SL + Hire 3                        02:03   
                            90%          Enforce SL        3                        02:07   
                                         Enforce SL + Hire 3                        01:57   
                            100%         Enforce SL        3                        02:05   
                                         Enforce SL + Hire 3                        01:35   

                                                                                           \
                                                                                            
                                                                                 Violated   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                        04:34   
                                         Enforce SL        3                        04:06   
                                         Enforce SL + Hire 3                            -   
                            80%          Enforce SL        3                        04:13   
                                         Enforce SL + Hire 3                        06:01   
                            90%          Enforce SL        3                        04:10   
                                         Enforce SL + Hire 3                        05:54   
                            100%         Enforce SL        3                        04:06   
                                         Enforce SL + Hire 3                     

### Table delay (latex)

In [11]:
print(file_util.df_to_latex(df_delay))

\begin{tabular}{lllllllllllllllll}
\toprule
   &             &      &                   &   & \multicolumn{4}{l}{Business} & \multicolumn{4}{l}{Standard} & \multicolumn{4}{l}{Low-cost} \\
   &             &      &                   &   & \multicolumn{2}{l}{Company} & \multicolumn{2}{l}{Freelance} & \multicolumn{2}{l}{Company} & \multicolumn{2}{l}{Freelance} & \multicolumn{2}{l}{Company} & \multicolumn{2}{l}{Freelance} \\
   &             &      &                   &   & Achieved & Violated &  Achieved & Violated & Achieved & Violated &  Achieved & Violated & Achieved & Violated &  Achieved & Violated \\
User base & Contract duration & Service rate & Policy & Max. hiring delay (s) &          &          &           &          &          &          &           &          &          &          &           &          \\
\midrule
\multirow{9}{*}{B+} & \multirow{9}{*}{Single-ride} & \multirow{3}{*}{0\%} & Min. waiting & 3 &    01:24 &    04:34 &         - &        - &    02:25 &    06:01 &   

### Table count

In [12]:
df_count = create_table(instance_attribute_count)
df_count

Business  \
                                                                                  Company   
                                                                                 Achieved   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                         1408   
                                         Enforce SL        3                         3772   
                                         Enforce SL + Hire 3                         6445   
                            80%          Enforce SL        3                         3695   
                                         Enforce SL + Hire 3                         5837   
                            90%          Enforce SL        3                         3736   
                                         Enforce SL + Hire 3                         6193   
                            100%         Enforce SL        3                         3800   
                                         Enforce SL + Hire 3                         6457   
S+        Single-ride       0%           Min. waiting      3                          590   
                                         Enforce SL        3                         2040   
                                         Enforce SL + Hire 3                         1753   
                            80%          Enforce SL        3                         1958   
                                         Enforce SL + Hire 3                         1717   
                            90%          Enforce SL        3                         2021   
                                         Enforce SL + Hire 3                         1781   
                            100%         Enforce SL        3                         2056   
                                         Enforce SL + Hire 3                         1744   
L+        Single-ride       0%           Min. waiting      3                          578   
                                         Enforce SL        3                         1955   
                                         Enforce SL + Hire 3                         1780   
                            80%          Enforce SL        3                         1908   
                                         Enforce SL + Hire 3                         1881   
                            90%          Enforce SL        3                         1921   
                                         Enforce SL + Hire 3                         1829   
                            100%         Enforce SL        3                         1946   
                                         Enforce SL + Hire 3                         1757   

                                                                                           \
                                                                                            
                                                                                 Violated   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                         8258   
                                         Enforce SL        3                         5894   
                                         Enforce SL + Hire 3                            0   
                            80%          Enforce SL        3                         5971   
                                         Enforce SL + Hire 3                          956   
                            90%          Enforce SL        3                         5930   
                                         Enforce SL + Hire 3                          462   
                            100%         Enforce SL        3                         5866   
                                         Enforce SL + Hire 3                     

### Table count (latex)

In [13]:
print(file_util.df_to_latex(df_count))

\begin{tabular}{lllllrrrrrrrrrrrrrrrrrr}
\toprule
   &             &      &                   &   & \multicolumn{6}{l}{Business} & \multicolumn{6}{l}{Standard} & \multicolumn{6}{l}{Low-cost} \\
   &             &      &                   &   & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} \\
   &             &      &                   &   & Achieved & Violated & Total &  Achieved & Violated & Total & Achieved & Violated & Total &  Achieved & Violated & Total & Achieved & Violated & Total &  Achieved & Violated & Total \\
User base & Contract duration & Service rate & Policy & Max. hiring delay (s) &          &          &       &           &          &       &          &          &       &           &          &       &          &          &       &           &          &       \\
\midrule
\multirow{9}{*}{B+} & \multirow{9}{*}{Single-ride} & \multirow

### Table percentage

In [14]:
df_percentage = create_table(instance_attribute_count_percentage)
df_percentage

Business  \
                                                                                   Company   
                                                                                  Achieved   
User base Contract duration Service rate Policy            Max. hiring delay (s)             
B+        Single-ride       0%           Min. waiting      3                      0.145665   
                                         Enforce SL        3                      0.390234   
                                         Enforce SL + Hire 3                      0.666770   
                            80%          Enforce SL        3                      0.382268   
                                         Enforce SL + Hire 3                      0.603869   
                            90%          Enforce SL        3                      0.386509   
                                         Enforce SL + Hire 3                      0.640699   
                            100%         Enforce SL        3                      0.393131   
                                         Enforce SL + Hire 3                      0.668012   
S+        Single-ride       0%           Min. waiting      3                      0.259113   
                                         Enforce SL        3                      0.895916   
                                         Enforce SL + Hire 3                      0.769873   
                            80%          Enforce SL        3                      0.859903   
                                         Enforce SL + Hire 3                      0.754062   
                            90%          Enforce SL        3                      0.887571   
                                         Enforce SL + Hire 3                      0.782170   
                            100%         Enforce SL        3                      0.902942   
                                         Enforce SL + Hire 3                      0.765920   
L+        Single-ride       0%           Min. waiting      3                      0.253843   
                                         Enforce SL        3                      0.858586   
                                         Enforce SL + Hire 3                      0.781730   
                            80%          Enforce SL        3                      0.837945   
                                         Enforce SL + Hire 3                      0.826087   
                            90%          Enforce SL        3                      0.843654   
                                         Enforce SL + Hire 3                      0.803250   
                            100%         Enforce SL        3                      0.854633   
                                         Enforce SL + Hire 3                      0.771629   

                                                                                            \
                                                                                             
                                                                                  Violated   
User base Contract duration Service rate Policy            Max. hiring delay (s)             
B+        Single-ride       0%           Min. waiting      3                      0.854335   
                                         Enforce SL        3                      0.609766   
                                         Enforce SL + Hire 3                      0.000000   
                            80%          Enforce SL        3                      0.617732   
                                         Enforce SL + Hire 3                      0.098903   
                            90%          Enforce SL        3                      0.613491   
                                         Enforce SL + Hire 3                      0.047796   
                            100%         Enforce SL        3                      0.606869   
                                       

### Table percentage (formatted)

In [15]:
df_percentage = df_util.format_as_percentage(df_percentage, cases=1)
df_percentage

Business  \
                                                                                  Company   
                                                                                 Achieved   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                        14.6%   
                                         Enforce SL        3                        39.0%   
                                         Enforce SL + Hire 3                        66.7%   
                            80%          Enforce SL        3                        38.2%   
                                         Enforce SL + Hire 3                        60.4%   
                            90%          Enforce SL        3                        38.7%   
                                         Enforce SL + Hire 3                        64.1%   
                            100%         Enforce SL        3                        39.3%   
                                         Enforce SL + Hire 3                        66.8%   
S+        Single-ride       0%           Min. waiting      3                        25.9%   
                                         Enforce SL        3                        89.6%   
                                         Enforce SL + Hire 3                        77.0%   
                            80%          Enforce SL        3                        86.0%   
                                         Enforce SL + Hire 3                        75.4%   
                            90%          Enforce SL        3                        88.8%   
                                         Enforce SL + Hire 3                        78.2%   
                            100%         Enforce SL        3                        90.3%   
                                         Enforce SL + Hire 3                        76.6%   
L+        Single-ride       0%           Min. waiting      3                        25.4%   
                                         Enforce SL        3                        85.9%   
                                         Enforce SL + Hire 3                        78.2%   
                            80%          Enforce SL        3                        83.8%   
                                         Enforce SL + Hire 3                        82.6%   
                            90%          Enforce SL        3                        84.4%   
                                         Enforce SL + Hire 3                        80.3%   
                            100%         Enforce SL        3                        85.5%   
                                         Enforce SL + Hire 3                        77.2%   

                                                                                           \
                                                                                            
                                                                                 Violated   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                        85.4%   
                                         Enforce SL        3                        61.0%   
                                         Enforce SL + Hire 3                            -   
                            80%          Enforce SL        3                        61.8%   
                                         Enforce SL + Hire 3                         9.9%   
                            90%          Enforce SL        3                        61.3%   
                                         Enforce SL + Hire 3                         4.8%   
                            100%         Enforce SL        3                        60.7%   
                                         Enforce SL + Hire 3                     

### Table percentage (latex)

In [16]:
print(file_util.df_to_latex(df_percentage))

\begin{tabular}{lllllllllllllllllllllll}
\toprule
   &             &      &                   &   & \multicolumn{6}{l}{Business} & \multicolumn{6}{l}{Standard} & \multicolumn{6}{l}{Low-cost} \\
   &             &      &                   &   & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} \\
   &             &      &                   &   & Achieved & Violated &   Total &  Achieved & Violated &  Total & Achieved & Violated &   Total &  Achieved & Violated & Total & Achieved & Violated &   Total &  Achieved & Violated & Total \\
User base & Contract duration & Service rate & Policy & Max. hiring delay (s) &          &          &         &           &          &        &          &          &         &           &          &       &          &          &         &           &          &       \\
\midrule
\multirow{9}{*}{B+} & \multirow{9}{*}{Single-rid

In [17]:
fleet = list(instance_case.dict_fleet.values())
fleet.remove("Denied")
tiers = list(instance_case.dict_sl_tier.values())
tiers.remove("Rejected")
classes = list(instance_case.dict_sl_class.values())

tuples = [
        (a, b, c) for a in classes
        for b in fleet for c in tiers
]

# Removing total
no_total= df_percentage[tuples]
no_total

Business  \
                                                                                  Company   
                                                                                 Achieved   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                        14.6%   
                                         Enforce SL        3                        39.0%   
                                         Enforce SL + Hire 3                        66.7%   
                            80%          Enforce SL        3                        38.2%   
                                         Enforce SL + Hire 3                        60.4%   
                            90%          Enforce SL        3                        38.7%   
                                         Enforce SL + Hire 3                        64.1%   
                            100%         Enforce SL        3                        39.3%   
                                         Enforce SL + Hire 3                        66.8%   
S+        Single-ride       0%           Min. waiting      3                        25.9%   
                                         Enforce SL        3                        89.6%   
                                         Enforce SL + Hire 3                        77.0%   
                            80%          Enforce SL        3                        86.0%   
                                         Enforce SL + Hire 3                        75.4%   
                            90%          Enforce SL        3                        88.8%   
                                         Enforce SL + Hire 3                        78.2%   
                            100%         Enforce SL        3                        90.3%   
                                         Enforce SL + Hire 3                        76.6%   
L+        Single-ride       0%           Min. waiting      3                        25.4%   
                                         Enforce SL        3                        85.9%   
                                         Enforce SL + Hire 3                        78.2%   
                            80%          Enforce SL        3                        83.8%   
                                         Enforce SL + Hire 3                        82.6%   
                            90%          Enforce SL        3                        84.4%   
                                         Enforce SL + Hire 3                        80.3%   
                            100%         Enforce SL        3                        85.5%   
                                         Enforce SL + Hire 3                        77.2%   

                                                                                           \
                                                                                            
                                                                                 Violated   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                        85.4%   
                                         Enforce SL        3                        61.0%   
                                         Enforce SL + Hire 3                            -   
                            80%          Enforce SL        3                        61.8%   
                                         Enforce SL + Hire 3                         9.9%   
                            90%          Enforce SL        3                        61.3%   
                                         Enforce SL + Hire 3                         4.8%   
                            100%         Enforce SL        3                        60.7%   
                                         Enforce SL + Hire 3                     

In [18]:
tuples

[('Business', 'Company', 'Achieved'),
 ('Business', 'Company', 'Violated'),
 ('Business', 'Freelance', 'Achieved'),
 ('Business', 'Freelance', 'Violated'),
 ('Standard', 'Company', 'Achieved'),
 ('Standard', 'Company', 'Violated'),
 ('Standard', 'Freelance', 'Achieved'),
 ('Standard', 'Freelance', 'Violated'),
 ('Low-cost', 'Company', 'Achieved'),
 ('Low-cost', 'Company', 'Violated'),
 ('Low-cost', 'Freelance', 'Achieved'),
 ('Low-cost', 'Freelance', 'Violated')]

In [19]:
tuples_no_freelance_violated = [
        (a, b, c) for a in classes
        for b in fleet for c in tiers
        if not(b == instance_case.dict_fleet["FREELANCE"] and c == instance_case.dict_sl_tier["SECOND"])
]

In [20]:
no_freelance_violated = no_total.copy()

no_freelance_violated = no_freelance_violated[tuples_no_freelance_violated]
no_freelance_violated

Business  \
                                                                                  Company   
                                                                                 Achieved   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                        14.6%   
                                         Enforce SL        3                        39.0%   
                                         Enforce SL + Hire 3                        66.7%   
                            80%          Enforce SL        3                        38.2%   
                                         Enforce SL + Hire 3                        60.4%   
                            90%          Enforce SL        3                        38.7%   
                                         Enforce SL + Hire 3                        64.1%   
                            100%         Enforce SL        3                        39.3%   
                                         Enforce SL + Hire 3                        66.8%   
S+        Single-ride       0%           Min. waiting      3                        25.9%   
                                         Enforce SL        3                        89.6%   
                                         Enforce SL + Hire 3                        77.0%   
                            80%          Enforce SL        3                        86.0%   
                                         Enforce SL + Hire 3                        75.4%   
                            90%          Enforce SL        3                        88.8%   
                                         Enforce SL + Hire 3                        78.2%   
                            100%         Enforce SL        3                        90.3%   
                                         Enforce SL + Hire 3                        76.6%   
L+        Single-ride       0%           Min. waiting      3                        25.4%   
                                         Enforce SL        3                        85.9%   
                                         Enforce SL + Hire 3                        78.2%   
                            80%          Enforce SL        3                        83.8%   
                                         Enforce SL + Hire 3                        82.6%   
                            90%          Enforce SL        3                        84.4%   
                                         Enforce SL + Hire 3                        80.3%   
                            100%         Enforce SL        3                        85.5%   
                                         Enforce SL + Hire 3                        77.2%   

                                                                                           \
                                                                                            
                                                                                 Violated   
User base Contract duration Service rate Policy            Max. hiring delay (s)            
B+        Single-ride       0%           Min. waiting      3                        85.4%   
                                         Enforce SL        3                        61.0%   
                                         Enforce SL + Hire 3                            -   
                            80%          Enforce SL        3                        61.8%   
                                         Enforce SL + Hire 3                         9.9%   
                            90%          Enforce SL        3                        61.3%   
                                         Enforce SL + Hire 3                         4.8%   
                            100%         Enforce SL        3                        60.7%   
                                         Enforce SL + Hire 3                     

In paper, adopted only a single maximal hiring delay = 3.

In [21]:
no_freelance_violated_single_hiring_delay = no_freelance_violated.reset_index()
no_freelance_violated_single_hiring_delay[instance_case.headers["maximal_hiring_delay"]]

0     3
1     3
2     3
3     3
4     3
5     3
6     3
7     3
8     3
9     3
10    3
11    3
12    3
13    3
14    3
15    3
16    3
17    3
18    3
19    3
20    3
21    3
22    3
23    3
24    3
25    3
26    3
Name: Max. hiring delay (s), dtype: object

In [22]:
filter_max_hiring_delay = no_freelance_violated_single_hiring_delay[instance_case.headers["maximal_hiring_delay"]].isin(["-", "--", "3"])
no_freelance_violated_single_hiring_delay = no_freelance_violated_single_hiring_delay[filter_max_hiring_delay]
no_freelance_violated_single_hiring_delay

User base Contract duration Service rate             Policy  \
                                                                 
                                                                 
0         B+       Single-ride           0%       Min. waiting   
1         B+       Single-ride           0%         Enforce SL   
2         B+       Single-ride           0%  Enforce SL + Hire   
3         B+       Single-ride          80%         Enforce SL   
4         B+       Single-ride          80%  Enforce SL + Hire   
5         B+       Single-ride          90%         Enforce SL   
6         B+       Single-ride          90%  Enforce SL + Hire   
7         B+       Single-ride         100%         Enforce SL   
8         B+       Single-ride         100%  Enforce SL + Hire   
9         S+       Single-ride           0%       Min. waiting   
10        S+       Single-ride           0%         Enforce SL   
11        S+       Single-ride           0%  Enforce SL + Hire   
12        S+       Single-ride          80%         Enforce SL   
13        S+       Single-ride          80%  Enforce SL + Hire   
14        S+       Single-ride          90%         Enforce SL   
15        S+       Single-ride          90%  Enforce SL + Hire   
16        S+       Single-ride         100%         Enforce SL   
17        S+       Single-ride         100%  Enforce SL + Hire   
18        L+       Single-ride           0%       Min. waiting   
19        L+       Single-ride           0%         Enforce SL   
20        L+       Single-ride           0%  Enforce SL + Hire   
21        L+       Single-ride          80%         Enforce SL   
22        L+       Single-ride          80%  Enforce SL + Hire   
23        L+       Single-ride          90%         Enforce SL   
24        L+       Single-ride          90%  Enforce SL + Hire   
25        L+       Single-ride         100%         Enforce SL   
26        L+       Single-ride         100%  Enforce SL + Hire   

   Max. hiring delay (s) Business                    Standard           \
                          Company          Freelance  Company            
                         Achieved Violated  Achieved Achieved Violated   
0                      3    14.6%    85.4%         -    73.3%    26.7%   
1                      3    39.0%    61.0%         -    89.9%    10.1%   
2                      3    66.7%        -     33.3%    99.3%        -   
3                      3    38.2%    61.8%         -    87.1%    12.9%   
4                      3    60.4%     9.9%     29.7%    92.1%     7.9%   
5                      3    38.7%    61.3%         -    87.5%    12.5%   
6                      3    64.1%     4.8%     31.2%    94.8%     5.2%   
7                      3    39.3%    60.7%         -    89.4%    10.6%   
8                      3    66.8%        -     33.2%    99.4%        -   
9                      3    25.9%    74.1%         -    56.0%    44.0%   
10                     3    89.6%    10.4%         -    83.4%    16.6%   
11                     3    77.0%        -     23.0%    97.4%        -   
12                     3    86.0%    14.0%         -    82.9%    17.1%   
13                     3    75.4%     7.8%     16.8%    88.4%    11.3%   
14                     3    88.8%    11.2%         -    82.3%    17.7%   
15                     3    78.2%     3.7%     18.1%    92.8%     5.6%   
16                     3    90.3%     9.7%         -    83.1%    16.9%   
17                     3    76.6%        -     23.4%    97.9%        -   
18                     3    25.4%    74.6%         -    61.8%    38.2%   
19                     3    85.9%    14.1%         -    91.2%     8.8%   
20                     3    78.2%        -     21.8%    96.0%        -   
21                     3    83.8%    16.2%         -    87.3%    12.7%   
22                     3    82.6%     8.4%      9.0%    88.4%    11.6%   
23                     3    84.4%    15.6%         -    89.0%    11.0%   
24                     3

In [23]:
columns = list(no_freelance_violated_single_hiring_delay.columns)
columns.remove((instance_case.headers["maximal_hiring_delay"], "", ""))
columns.remove((instance_case.headers["contract_duration"], "", ""))
columns

[('User base', '', ''),
 ('Service rate', '', ''),
 ('Policy', '', ''),
 ('Business', 'Company', 'Achieved'),
 ('Business', 'Company', 'Violated'),
 ('Business', 'Freelance', 'Achieved'),
 ('Standard', 'Company', 'Achieved'),
 ('Standard', 'Company', 'Violated'),
 ('Standard', 'Freelance', 'Achieved'),
 ('Low-cost', 'Company', 'Achieved'),
 ('Low-cost', 'Company', 'Violated'),
 ('Low-cost', 'Freelance', 'Achieved')]

In [24]:
no_freelance_violated_single_hiring_delay[('space1', '', '')] = ""
no_freelance_violated_single_hiring_delay[('space2', '', '')] = ""
no_freelance_violated_single_hiring_delay[('space3', '', '')] = ""

columns = [('User base', '', ''),
 ('Service rate', '', ''),
 ('Policy', '', ''),
           ('space1', '', ''),
 ('Business', 'Company', 'Achieved'),
 ('Business', 'Freelance', 'Achieved'),
 ('Business', 'Company', 'Violated'),
           ('space2', '', ''),
 ('Standard', 'Company', 'Achieved'),
 ('Standard', 'Freelance', 'Achieved'),
 ('Standard', 'Company', 'Violated'),
           ('space3', '', ''),
 ('Low-cost', 'Company', 'Achieved'),
 ('Low-cost', 'Freelance', 'Achieved'),
 ('Low-cost', 'Company', 'Violated')]

In [25]:
no_freelance_violated_single_hiring_delay = no_freelance_violated_single_hiring_delay[columns]
no_freelance_violated_single_hiring_delay = no_freelance_violated_single_hiring_delay.set_index(
    [('User base', '', ''),
     ('Service rate', '', ''),
 ('Policy', '', '')])

In [26]:
print(no_freelance_violated_single_hiring_delay.columns)

MultiIndex([(  'space1',          '',         ''),
            ('Business',   'Company', 'Achieved'),
            ('Business', 'Freelance', 'Achieved'),
            ('Business',   'Company', 'Violated'),
            (  'space2',          '',         ''),
            ('Standard',   'Company', 'Achieved'),
            ('Standard', 'Freelance', 'Achieved'),
            ('Standard',   'Company', 'Violated'),
            (  'space3',          '',         ''),
            ('Low-cost',   'Company', 'Achieved'),
            ('Low-cost', 'Freelance', 'Achieved'),
            ('Low-cost',   'Company', 'Violated')],
           )


In [27]:
no_freelance_violated_single_hiring_delay.index.names = [
        instance_case.headers["user_base"],
        instance_case.headers["service_rate"],
        instance_case.headers["method"],
    ]
no_freelance_violated_single_hiring_delay

space1 Business                     \
                                                 Company Freelance  Company   
                                                Achieved  Achieved Violated   
User base Service rate Policy                                                 
B+        0%           Min. waiting                14.6%         -    85.4%   
                       Enforce SL                  39.0%         -    61.0%   
                       Enforce SL + Hire           66.7%     33.3%        -   
          80%          Enforce SL                  38.2%         -    61.8%   
                       Enforce SL + Hire           60.4%     29.7%     9.9%   
          90%          Enforce SL                  38.7%         -    61.3%   
                       Enforce SL + Hire           64.1%     31.2%     4.8%   
          100%         Enforce SL                  39.3%         -    60.7%   
                       Enforce SL + Hire           66.8%     33.2%        -   
S+        0%           Min. waiting                25.9%         -    74.1%   
                       Enforce SL                  89.6%         -    10.4%   
                       Enforce SL + Hire           77.0%     23.0%        -   
          80%          Enforce SL                  86.0%         -    14.0%   
                       Enforce SL + Hire           75.4%     16.8%     7.8%   
          90%          Enforce SL                  88.8%         -    11.2%   
                       Enforce SL + Hire           78.2%     18.1%     3.7%   
          100%         Enforce SL                  90.3%         -     9.7%   
                       Enforce SL + Hire           76.6%     23.4%        -   
L+        0%           Min. waiting                25.4%         -    74.6%   
                       Enforce SL                  85.9%         -    14.1%   
                       Enforce SL + Hire           78.2%     21.8%        -   
          80%          Enforce SL                  83.8%         -    16.2%   
                       Enforce SL + Hire           82.6%      9.0%     8.4%   
          90%          Enforce SL                  84.4%         -    15.6%   
                       Enforce SL + Hire           80.3%     15.9%     3.7%   
          100%         Enforce SL                  85.5%         -    14.5%   
                       Enforce SL + Hire           77.2%     22.8%        -   

                                         space2 Standard                     \
                                                 Company Freelance  Company   
                                                Achieved  Achieved Violated   
User base Service rate Policy                                                 
B+        0%           Min. waiting                73.3%         -    26.7%   
                       Enforce SL                  89.9%         -    10.1%   
                       Enforce SL + Hire           99.3%         *        -   
          80%          Enforce SL                  87.1%         -    12.9%   
                       Enforce SL + Hire           92.1%         -     7.9%   
          90%          Enforce SL                  87.5%         -    12.5%   
                       Enforce SL + Hire           94.8%         *     5.2%   
          100%         Enforce SL                  89.4%         -    10.6%   
                       Enforce SL + Hire           99.4%         *        -   
S+        0%           Min. waiting                56.0%         -    44.0%   
                       Enforce SL                  83.4%         -    16.6%   
                       Enforce SL + Hire           97.4%      2.6%        -   
          80%          Enforce SL                  82.9%         -    17.1%   
                       Enforce SL + Hire           88.4%         *    11.3%   
          90%          Enforce SL                  82.3%         -    17.7%   
                       Enforce SL + Hire           92.8%      1.6%     5.6%   
          1

In [28]:
print(file_util.df_to_latex(no_freelance_violated_single_hiring_delay))


\begin{tabular}{lllllllllllllll}
\toprule
   &      &                   & space1 & \multicolumn{3}{l}{Business} & space2 & \multicolumn{3}{l}{Standard} & space3 & \multicolumn{3}{l}{Low-cost} \\
   &      &                   &  Company & Freelance & \multicolumn{2}{l}{Company} &  Company & Freelance & \multicolumn{2}{l}{Company} &  Company & Freelance &  Company \\
   &      &                   & Achieved &  Achieved & \multicolumn{2}{l}{Violated} & Achieved &  Achieved & \multicolumn{2}{l}{Violated} & Achieved &  Achieved & Violated \\
User base & Service rate & Policy &        &          &           &          &        &          &           &          &        &          &           &          \\
\midrule
\multirow{9}{*}{B+} & \multirow{3}{*}{0\%} & Min. waiting &        &    14.6\% &         - &    85.4\% &        &    73.3\% &         - &    26.7\% &        &    98.3\% &         - &     1.7\% \\
   &      & Enforce SL &        &    39.0\% &         - &    61.0\% &        &    89.9